## 53 - Xarray Basics

Watch video: [YouTube](https://www.youtube.com/watch?v=_9j7Y1-lk-o)

In [10]:
import datetime
import numpy as np
import xarray as xr
from siphon.catalog import TDSCatalog
from xarray.backends import NetCDF4DataStore
from metpy.testing import get_test_data

### Input / Output

In [11]:
# An example method for reading in data: local NetCDF file
# Subset GFS output
ds = xr.open_dataset(get_test_data('irma_gfs_example.nc', False))
ds

<xarray.Dataset> Size: 67MB
Dimensions:                              (time1: 9, latitude: 81,
                                          isobaric3: 31, isobaric1: 21,
                                          longitude: 131)
Coordinates:
  * time1                                (time1) datetime64[ns] 72B 2017-09-0...
    reftime                              datetime64[ns] 8B ...
  * latitude                             (latitude) float32 324B 50.0 ... 10.0
  * isobaric3                            (isobaric3) float64 248B 100.0 ... 1...
  * isobaric1                            (isobaric1) float64 168B 1e+04 ... 1...
  * longitude                            (longitude) float32 524B 250.0 ... 3...
Data variables:
    Vertical_velocity_pressure_isobaric  (time1, isobaric1, latitude, longitude) float32 8MB ...
    Relative_humidity_isobaric           (time1, isobaric3, latitude, longitude) float32 12MB ...
    Temperature_isobaric                 (time1, isobaric3, latitude, longitude) float32 12MB ...
    u-component_of_wind_isobaric         (time1, isobaric3, latitude, longitude) float32 12MB ...
    v-component_of_wind_isobaric         (time1, isobaric3, latitude, longitude) float32 12MB ...
    Geopotential_height_isobaric         (time1, isobaric3, latitude, longitude) float32 12MB ...
    LatLon_361X720-0p25S-180p00E         int32 4B ...
Attributes: (12/13)
    Originating_or_generating_Center:                                        ...
    Originating_or_generating_Subcenter:                                     ...
    GRIB_table_version:                                                      ...
    Type_of_generating_process:                                              ...
    Analysis_or_forecast_generating_process_identifier_defined_by_originating...
    Conventions:                                                             ...
    ...                                                                                ...
    featureType:                                                             ...
    History:                                                                 ...
    geospatial_lat_min:                                                      ...
    geospatial_lat_max:                                                      ...
    geospatial_lon_min:                                                      ...
    geospatial_lon_max:                                                      ...

In [13]:
# Another example method: OPENDaP
# NAM Analysis from NCEI NOMADS
# base_url = 'https://www.ncei.noaa.gov/thredds/dodsC/namanl'
url = 'https://www.ncei.noaa.gov/thredds/dodsC/model-namanl-old'
dt = datetime.datetime(2016, 4, 16, 18, 0, 0)
ds = xr.open_dataset(f'{url}{dt:%Y%m}/{dt:%Y%m%d}/namanl_218_{dt:%Y%m%d}_{dt:%H}00_000.grb', decode_times=True)
ds

<xarray.Dataset> Size: 468MB
Dimensions:                                                                                        (
                                                                                                    x: 614,
                                                                                                    y: 428,
                                                                                                    time: 1,
                                                                                                    time_bounds_1: 2,
                                                                                                    time1: 1,
                                                                                                    ...
                                                                                                    height_above_ground3: 2,
                                                                                                    layer_between_two_depths_below_surface_layer2: 1,
                                                                                                    layer_between_two_depths_below_surface_layer2_bounds_1: 2,
                                                                                                    height_above_ground4: 2,
                                                                                                    layer_between_two_sigmas_layer: 1,
                                                                                                    layer_between_two_sigmas_layer_bounds_1: 2)
Coordinates: (12/25)
  * x                                                                                              (x) float32 2kB ...
  * y                                                                                              (y) float32 2kB ...
    reftime                                                                                        datetime64[ns] 8B ...
  * time                                                                                           (time) datetime64[ns] 8B ...
  * time1                                                                                          (time1) datetime64[ns] 8B ...
  * layer_between_two_heights_above_ground_layer                                                   (layer_between_two_heights_above_ground_layer) float32 4B ...
    ...                                                                                             ...
  * hybrid                                                                                         (hybrid) float32 4B ...
  * layer_between_two_pressure_difference_from_ground_layer2                                       (layer_between_two_pressure_difference_from_ground_layer2) float32 4B ...
  * height_above_ground3                                                                           (height_above_ground3) float32 8B ...
  * layer_between_two_depths_below_surface_layer2                                                  (layer_between_two_depths_below_surface_layer2) float32 4B ...
  * height_above_ground4                                                                           (height_above_ground4) float32 8B ...
  * layer_between_two_sigmas_layer                                                                 (layer_between_two_sigmas_layer) float32 4B ...
Dimensions without coordinates: time_bounds_1,
                                layer_between_two_heights_above_ground_layer_bounds_1,
                                layer_between_two_pressure_difference_from_ground_layer_bounds_1,
                                layer_between_two_depths_below_surface_layer_bounds_1,
                                layer_between_two_pressure_difference_from_ground_layer1_bounds_1,
                                layer_between_two_isobaric_layer_bounds_1,
                                layer_between_two_heights_above_ground_layer1_bounds_1,
             

In [14]:
# Yet another example method: siphon
# GOES16 Satellite
date = datetime.datetime.utcnow()
channel = 8
region = 'CONUS'

In [15]:
url = f'https://thredds-test.unidata.ucar.edu/thredds/catalog/satellite/goes/east/grb/ABI/{region}/Channel{channel:02d}/{date:%Y%m%d}/catalog.xml'
cat = TDSCatalog(url)
ds = cat.datasets[-2]
ds = ds.remote_access(service = 'OPENDAP')
ds = xr.open_dataset(NetCDF4DataStore(ds))
ds

<xarray.Dataset> Size: 30MB
Dimensions:                                           (y: 1500, x: 2500,
                                                       number_of_time_bounds: 2,
                                                       number_of_image_bounds: 2,
                                                       band: 1,
                                                       number_of_harmonization_coefficients: 3,
                                                       num_star_looks: 24)
Coordinates:
  * y                                                 (y) float64 12kB 0.1282...
  * x                                                 (x) float64 20kB -0.101...
    t                                                 datetime64[ns] 8B ...
    y_image                                           float32 4B ...
    x_image                                           float32 4B ...
    band_id                                           (band) int8 1B ...
    band_wavelength                                   (band) float32 4B ...
    t_star_look                                       (num_star_looks) datetime64[ns] 192B ...
    band_wavelength_star_look                         (num_star_looks) float32 96B ...
Dimensions without coordinates: number_of_time_bounds, number_of_image_bounds,
                                band, number_of_harmonization_coefficients,
                                num_star_looks
Data variables: (12/39)
    time_bounds                                       (number_of_time_bounds) datetime64[ns] 16B ...
    goes_imager_projection                            int32 4B ...
    y_image_bounds                                    (number_of_image_bounds) float32 8B ...
    x_image_bounds                                    (number_of_image_bounds) float32 8B ...
    nominal_satellite_subpoint_lat                    float32 4B ...
    nominal_satellite_subpoint_lon                    float32 4B ...
    ...                                                ...
    algorithm_product_version_container               int32 4B ...
    star_id                                           (num_star_looks) float32 96B ...
    channel_integration_time                          float64 8B ...
    channel_gain_field                                float64 8B ...
    Rad                                               (y, x) float32 15MB ...
    DQF                                               (y, x) float32 15MB ...
Attributes: (12/33)
    naming_authority:                      gov.nesdis.noaa
    Conventions:                           CF-1.7
    Metadata_Conventions:                  Unidata Dataset Discovery v1.0
    standard_name_vocabulary:              CF Standard Name Table (v35, 20 Ju...
    institution:                           DOC/NOAA/NESDIS > U.S. Department ...
    project:                               GOES
    ...                                    ...
    time_coverage_end:                     2024-03-26T05:53:52.7Z
    LUT_Filenames:                         SpaceLookParams(FM1A_CDRL79RevH_DO...
    cspp_geo_grb_version:                  CSPP Geo GRB v1.0.27
    cspp_geo_grb_production_host:          ingest.unidata.ucar.edu
    history:                               2024-03-26, 05:54:05, ldm, /usr/lo...
    cspp_geo_grb_reconstruction_end_time:  2024-03-26 05:54:05.816580

In [16]:
# Creating a DataArray
index = xr.DataArray(range(7), name='index', dims=['index'])
index

<xarray.DataArray 'index' (index: 7)> Size: 28B
array([0, 1, 2, 3, 4, 5, 6])
Dimensions without coordinates: index

In [17]:
xr.DataArray(np.linspace(1000, 500, 5), name='isobaric', dims=['isobaric'], attrs={'units': 'hPa'})

<xarray.DataArray 'isobaric' (isobaric: 5)> Size: 40B
array([1000.,  875.,  750.,  625.,  500.])
Dimensions without coordinates: isobaric
Attributes:
    units:    hPa

In [18]:
# Creating a Dataset
data_temp = np.linspace(250, 300, 5 * 6 * 7).reshape((5, 6, 7))
data_rh = np.linspace(0, 1, 5 * 6 * 7).reshape((5, 6, 7))
ds = xr.Dataset(data_vars=dict(
                    temperature=(['isobaric', 'lat', 'lon'], data_temp), 
                    relative_humidity=(['isobaric', 'lat', 'lon'], data_rh)
                ),
                coords=dict(
                    isobaric=xr.DataArray(np.linspace(1000, 500, 5), name='isobaric', dims=['isobaric'], attrs={'units': 'hPa'}), 
                    lat=xr.DataArray(np.linspace(30, 45, 6), name='lat', dims=['lat'], attrs={'units': 'degrees_north'}), 
                    lon=xr.DataArray(np.linspace(255, 275, 7), name='lon', dims=['lon'], attrs={'units': 'degrees_east'})
                )
)
ds

<xarray.Dataset> Size: 4kB
Dimensions:            (isobaric: 5, lat: 6, lon: 7)
Coordinates:
  * isobaric           (isobaric) float64 40B 1e+03 875.0 750.0 625.0 500.0
  * lat                (lat) float64 48B 30.0 33.0 36.0 39.0 42.0 45.0
  * lon                (lon) float64 56B 255.0 258.3 261.7 ... 268.3 271.7 275.0
Data variables:
    temperature        (isobaric, lat, lon) float64 2kB 250.0 250.2 ... 300.0
    relative_humidity  (isobaric, lat, lon) float64 2kB 0.0 0.004785 ... 1.0

In [19]:
ds['isobaric']

<xarray.DataArray 'isobaric' (isobaric: 5)> Size: 40B
array([1000.,  875.,  750.,  625.,  500.])
Coordinates:
  * isobaric  (isobaric) float64 40B 1e+03 875.0 750.0 625.0 500.0
Attributes:
    units:    hPa

In [20]:
ds['temperature']

<xarray.DataArray 'temperature' (isobaric: 5, lat: 6, lon: 7)> Size: 2kB
array([[[250.        , 250.23923445, 250.4784689 , 250.71770335,
         250.9569378 , 251.19617225, 251.4354067 ],
        [251.67464115, 251.9138756 , 252.15311005, 252.3923445 ,
         252.63157895, 252.8708134 , 253.11004785],
        [253.3492823 , 253.58851675, 253.8277512 , 254.06698565,
         254.3062201 , 254.54545455, 254.784689  ],
        [255.02392344, 255.26315789, 255.50239234, 255.74162679,
         255.98086124, 256.22009569, 256.45933014],
        [256.69856459, 256.93779904, 257.17703349, 257.41626794,
         257.65550239, 257.89473684, 258.13397129],
        [258.37320574, 258.61244019, 258.85167464, 259.09090909,
         259.33014354, 259.56937799, 259.80861244]],

       [[260.04784689, 260.28708134, 260.52631579, 260.76555024,
         261.00478469, 261.24401914, 261.48325359],
        [261.72248804, 261.96172249, 262.20095694, 262.44019139,
         262.67942584, 262.91866029, 263.15789474],
        [263.39712919, 263.63636364, 263.87559809, 264.11483254,
         264.35406699, 264.59330144, 264.83253589],
        [265.07177033, 265.31100478, 265.55023923, 265.78947368,
...
         284.44976077, 284.68899522, 284.92822967],
        [285.16746411, 285.40669856, 285.64593301, 285.88516746,
         286.12440191, 286.36363636, 286.60287081],
        [286.84210526, 287.08133971, 287.32057416, 287.55980861,
         287.79904306, 288.03827751, 288.27751196],
        [288.51674641, 288.75598086, 288.99521531, 289.23444976,
         289.47368421, 289.71291866, 289.95215311]],

       [[290.19138756, 290.43062201, 290.66985646, 290.90909091,
         291.14832536, 291.38755981, 291.62679426],
        [291.86602871, 292.10526316, 292.34449761, 292.58373206,
         292.82296651, 293.06220096, 293.30143541],
        [293.54066986, 293.77990431, 294.01913876, 294.25837321,
         294.49760766, 294.73684211, 294.97607656],
        [295.215311  , 295.45454545, 295.6937799 , 295.93301435,
         296.1722488 , 296.41148325, 296.6507177 ],
        [296.88995215, 297.1291866 , 297.36842105, 297.6076555 ,
         297.84688995, 298.0861244 , 298.32535885],
        [298.5645933 , 298.80382775, 299.0430622 , 299.28229665,
         299.5215311 , 299.76076555, 300.        ]]])
Coordinates:
  * isobaric  (isobaric) float64 40B 1e+03 875.0 750.0 625.0 500.0
  * lat       (lat) float64 48B 30.0 33.0 36.0 39.0 42.0 45.0
  * lon       (lon) float64 56B 255.0 258.3 261.7 265.0 268.3 271.7 275.0

In [21]:
ds['temperature'].attrs['units'] = 'kelvin'
ds['relative_humidity'].attrs['units'] = 'dimentionless'

In [22]:
ds['temperature']

<xarray.DataArray 'temperature' (isobaric: 5, lat: 6, lon: 7)> Size: 2kB
array([[[250.        , 250.23923445, 250.4784689 , 250.71770335,
         250.9569378 , 251.19617225, 251.4354067 ],
        [251.67464115, 251.9138756 , 252.15311005, 252.3923445 ,
         252.63157895, 252.8708134 , 253.11004785],
        [253.3492823 , 253.58851675, 253.8277512 , 254.06698565,
         254.3062201 , 254.54545455, 254.784689  ],
        [255.02392344, 255.26315789, 255.50239234, 255.74162679,
         255.98086124, 256.22009569, 256.45933014],
        [256.69856459, 256.93779904, 257.17703349, 257.41626794,
         257.65550239, 257.89473684, 258.13397129],
        [258.37320574, 258.61244019, 258.85167464, 259.09090909,
         259.33014354, 259.56937799, 259.80861244]],

       [[260.04784689, 260.28708134, 260.52631579, 260.76555024,
         261.00478469, 261.24401914, 261.48325359],
        [261.72248804, 261.96172249, 262.20095694, 262.44019139,
         262.67942584, 262.91866029, 263.15789474],
        [263.39712919, 263.63636364, 263.87559809, 264.11483254,
         264.35406699, 264.59330144, 264.83253589],
        [265.07177033, 265.31100478, 265.55023923, 265.78947368,
...
         284.44976077, 284.68899522, 284.92822967],
        [285.16746411, 285.40669856, 285.64593301, 285.88516746,
         286.12440191, 286.36363636, 286.60287081],
        [286.84210526, 287.08133971, 287.32057416, 287.55980861,
         287.79904306, 288.03827751, 288.27751196],
        [288.51674641, 288.75598086, 288.99521531, 289.23444976,
         289.47368421, 289.71291866, 289.95215311]],

       [[290.19138756, 290.43062201, 290.66985646, 290.90909091,
         291.14832536, 291.38755981, 291.62679426],
        [291.86602871, 292.10526316, 292.34449761, 292.58373206,
         292.82296651, 293.06220096, 293.30143541],
        [293.54066986, 293.77990431, 294.01913876, 294.25837321,
         294.49760766, 294.73684211, 294.97607656],
        [295.215311  , 295.45454545, 295.6937799 , 295.93301435,
         296.1722488 , 296.41148325, 296.6507177 ],
        [296.88995215, 297.1291866 , 297.36842105, 297.6076555 ,
         297.84688995, 298.0861244 , 298.32535885],
        [298.5645933 , 298.80382775, 299.0430622 , 299.28229665,
         299.5215311 , 299.76076555, 300.        ]]])
Coordinates:
  * isobaric  (isobaric) float64 40B 1e+03 875.0 750.0 625.0 500.0
  * lat       (lat) float64 48B 30.0 33.0 36.0 39.0 42.0 45.0
  * lon       (lon) float64 56B 255.0 258.3 261.7 265.0 268.3 271.7 275.0
Attributes:
    units:    kelvin

In [23]:
# Writing to netCDF
ds.to_netcdf('fake_data.nc')

### Indexing and Selecting Data

In [24]:
ds = xr.open_dataset(get_test_data('irma_gfs_example.nc', False))
heights = ds['Geopotential_height_isobaric']
heights

<xarray.DataArray 'Geopotential_height_isobaric' (time1: 9, isobaric3: 31,
                                                  latitude: 81, longitude: 131)> Size: 12MB
[2960469 values with dtype=float32]
Coordinates:
  * time1      (time1) datetime64[ns] 72B 2017-09-05T12:00:00 ... 2017-09-06T...
    reftime    datetime64[ns] 8B ...
  * latitude   (latitude) float32 324B 50.0 49.5 49.0 48.5 ... 11.0 10.5 10.0
  * isobaric3  (isobaric3) float64 248B 100.0 200.0 300.0 ... 9.75e+04 1e+05
  * longitude  (longitude) float32 524B 250.0 250.5 251.0 ... 314.0 314.5 315.0
Attributes:
    long_name:                      Geopotential height @ Isobaric surface
    units:                          gpm
    Grib_Variable_Id:               VAR_0-3-5_L100
    Grib2_Parameter:                [0 3 5]
    Grib2_Parameter_Discipline:     Meteorological products
    Grib2_Parameter_Category:       Mass
    Grib2_Parameter_Name:           Geopotential height
    Grib2_Level_Type:               100
    Grib2_Level_Desc:               Isobaric surface
    Grib2_Generating_Process_Type:  Forecast
    grid_mapping:                   LatLon_361X720-0p25S-180p00E

In [25]:
# Standard NumPy indexing (integer based positional indexing)
heights[0, -1]

<xarray.DataArray 'Geopotential_height_isobaric' (latitude: 81, longitude: 131)> Size: 42kB
[10611 values with dtype=float32]
Coordinates:
    time1      datetime64[ns] 8B 2017-09-05T12:00:00
    reftime    datetime64[ns] 8B ...
  * latitude   (latitude) float32 324B 50.0 49.5 49.0 48.5 ... 11.0 10.5 10.0
    isobaric3  float64 8B 1e+05
  * longitude  (longitude) float32 524B 250.0 250.5 251.0 ... 314.0 314.5 315.0
Attributes:
    long_name:                      Geopotential height @ Isobaric surface
    units:                          gpm
    Grib_Variable_Id:               VAR_0-3-5_L100
    Grib2_Parameter:                [0 3 5]
    Grib2_Parameter_Discipline:     Meteorological products
    Grib2_Parameter_Category:       Mass
    Grib2_Parameter_Name:           Geopotential height
    Grib2_Level_Type:               100
    Grib2_Level_Desc:               Isobaric surface
    Grib2_Generating_Process_Type:  Forecast
    grid_mapping:                   LatLon_361X720-0p25S-180p00E

In [26]:
# Label-based positional indexing
heights.loc['2017-09-06T06:00', 85000]

<xarray.DataArray 'Geopotential_height_isobaric' (latitude: 81, longitude: 131)> Size: 42kB
[10611 values with dtype=float32]
Coordinates:
    time1      datetime64[ns] 8B 2017-09-06T06:00:00
    reftime    datetime64[ns] 8B ...
  * latitude   (latitude) float32 324B 50.0 49.5 49.0 48.5 ... 11.0 10.5 10.0
    isobaric3  float64 8B 8.5e+04
  * longitude  (longitude) float32 524B 250.0 250.5 251.0 ... 314.0 314.5 315.0
Attributes:
    long_name:                      Geopotential height @ Isobaric surface
    units:                          gpm
    Grib_Variable_Id:               VAR_0-3-5_L100
    Grib2_Parameter:                [0 3 5]
    Grib2_Parameter_Discipline:     Meteorological products
    Grib2_Parameter_Category:       Mass
    Grib2_Parameter_Name:           Geopotential height
    Grib2_Level_Type:               100
    Grib2_Level_Desc:               Isobaric surface
    Grib2_Generating_Process_Type:  Forecast
    grid_mapping:                   LatLon_361X720-0p25S-180p00E

In [27]:
# Integer-based dimension name indexing
heights.isel(time1=-1, isobaric3=14)

<xarray.DataArray 'Geopotential_height_isobaric' (latitude: 81, longitude: 131)> Size: 42kB
[10611 values with dtype=float32]
Coordinates:
    time1      datetime64[ns] 8B 2017-09-06T12:00:00
    reftime    datetime64[ns] 8B ...
  * latitude   (latitude) float32 324B 50.0 49.5 49.0 48.5 ... 11.0 10.5 10.0
    isobaric3  float64 8B 3e+04
  * longitude  (longitude) float32 524B 250.0 250.5 251.0 ... 314.0 314.5 315.0
Attributes:
    long_name:                      Geopotential height @ Isobaric surface
    units:                          gpm
    Grib_Variable_Id:               VAR_0-3-5_L100
    Grib2_Parameter:                [0 3 5]
    Grib2_Parameter_Discipline:     Meteorological products
    Grib2_Parameter_Category:       Mass
    Grib2_Parameter_Name:           Geopotential height
    Grib2_Level_Type:               100
    Grib2_Level_Desc:               Isobaric surface
    Grib2_Generating_Process_Type:  Forecast
    grid_mapping:                   LatLon_361X720-0p25S-180p00E

In [28]:
# Label-based dimension name indexing
heights.sel(time1='2017-09-06T06:00', latitude=30)


<xarray.DataArray 'Geopotential_height_isobaric' (isobaric3: 31, longitude: 131)> Size: 16kB
[4061 values with dtype=float32]
Coordinates:
    time1      datetime64[ns] 8B 2017-09-06T06:00:00
    reftime    datetime64[ns] 8B ...
    latitude   float32 4B 30.0
  * isobaric3  (isobaric3) float64 248B 100.0 200.0 300.0 ... 9.75e+04 1e+05
  * longitude  (longitude) float32 524B 250.0 250.5 251.0 ... 314.0 314.5 315.0
Attributes:
    long_name:                      Geopotential height @ Isobaric surface
    units:                          gpm
    Grib_Variable_Id:               VAR_0-3-5_L100
    Grib2_Parameter:                [0 3 5]
    Grib2_Parameter_Discipline:     Meteorological products
    Grib2_Parameter_Category:       Mass
    Grib2_Parameter_Name:           Geopotential height
    Grib2_Level_Type:               100
    Grib2_Level_Desc:               Isobaric surface
    Grib2_Generating_Process_Type:  Forecast
    grid_mapping:                   LatLon_361X720-0p25S-180p00E

In [29]:
# Label-based dimension name indexing, with nearest point search 
heights.sel(time1='2017-09-06T06:00', latitude=24.56, longitude=278.24, method='nearest')

<xarray.DataArray 'Geopotential_height_isobaric' (isobaric3: 31)> Size: 124B
[31 values with dtype=float32]
Coordinates:
    time1      datetime64[ns] 8B 2017-09-06T06:00:00
    reftime    datetime64[ns] 8B ...
    latitude   float32 4B 24.5
  * isobaric3  (isobaric3) float64 248B 100.0 200.0 300.0 ... 9.75e+04 1e+05
    longitude  float32 4B 278.0
Attributes:
    long_name:                      Geopotential height @ Isobaric surface
    units:                          gpm
    Grib_Variable_Id:               VAR_0-3-5_L100
    Grib2_Parameter:                [0 3 5]
    Grib2_Parameter_Discipline:     Meteorological products
    Grib2_Parameter_Category:       Mass
    Grib2_Parameter_Name:           Geopotential height
    Grib2_Level_Type:               100
    Grib2_Level_Desc:               Isobaric surface
    Grib2_Generating_Process_Type:  Forecast
    grid_mapping:                   LatLon_361X720-0p25S-180p00E

In [30]:
# Label-based dimension anme indexing, with nearest point search, on full dataset
ds.sel(time1='2017-09-06T06:00', latitude=24.56, longitude=278.24, method='nearest')

<xarray.Dataset> Size: 1kB
Dimensions:                              (isobaric3: 31, isobaric1: 21)
Coordinates:
    time1                                datetime64[ns] 8B 2017-09-06T06:00:00
    reftime                              datetime64[ns] 8B ...
    latitude                             float32 4B 24.5
  * isobaric3                            (isobaric3) float64 248B 100.0 ... 1...
  * isobaric1                            (isobaric1) float64 168B 1e+04 ... 1...
    longitude                            float32 4B 278.0
Data variables:
    Vertical_velocity_pressure_isobaric  (isobaric1) float32 84B ...
    Relative_humidity_isobaric           (isobaric3) float32 124B ...
    Temperature_isobaric                 (isobaric3) float32 124B ...
    u-component_of_wind_isobaric         (isobaric3) float32 124B ...
    v-component_of_wind_isobaric         (isobaric3) float32 124B ...
    Geopotential_height_isobaric         (isobaric3) float32 124B ...
    LatLon_361X720-0p25S-180p00E         int32 4B ...
Attributes: (12/13)
    Originating_or_generating_Center:                                        ...
    Originating_or_generating_Subcenter:                                     ...
    GRIB_table_version:                                                      ...
    Type_of_generating_process:                                              ...
    Analysis_or_forecast_generating_process_identifier_defined_by_originating...
    Conventions:                                                             ...
    ...                                                                                ...
    featureType:                                                             ...
    History:                                                                 ...
    geospatial_lat_min:                                                      ...
    geospatial_lat_max:                                                      ...
    geospatial_lon_min:                                                      ...
    geospatial_lon_max:                                                      ...

In [31]:
# Getting a range of times
heights.sel(time1=slice('2017-09-06T00:00', '2017-09-07T00:00'), isobaric3=50000, latitude=24.5, longitude=278)

<xarray.DataArray 'Geopotential_height_isobaric' (time1: 5)> Size: 20B
[5 values with dtype=float32]
Coordinates:
  * time1      (time1) datetime64[ns] 40B 2017-09-06 ... 2017-09-06T12:00:00
    reftime    datetime64[ns] 8B ...
    latitude   float32 4B 24.5
    isobaric3  float64 8B 5e+04
    longitude  float32 4B 278.0
Attributes:
    long_name:                      Geopotential height @ Isobaric surface
    units:                          gpm
    Grib_Variable_Id:               VAR_0-3-5_L100
    Grib2_Parameter:                [0 3 5]
    Grib2_Parameter_Discipline:     Meteorological products
    Grib2_Parameter_Category:       Mass
    Grib2_Parameter_Name:           Geopotential height
    Grib2_Level_Type:               100
    Grib2_Level_Desc:               Isobaric surface
    Grib2_Generating_Process_Type:  Forecast
    grid_mapping:                   LatLon_361X720-0p25S-180p00E

In [32]:
heights[{'longitude': heights['longitude'] > 270}]

<xarray.DataArray 'Geopotential_height_isobaric' (time1: 9, isobaric3: 31,
                                                  latitude: 81, longitude: 90)> Size: 8MB
[2033910 values with dtype=float32]
Coordinates:
  * time1      (time1) datetime64[ns] 72B 2017-09-05T12:00:00 ... 2017-09-06T...
    reftime    datetime64[ns] 8B ...
  * latitude   (latitude) float32 324B 50.0 49.5 49.0 48.5 ... 11.0 10.5 10.0
  * isobaric3  (isobaric3) float64 248B 100.0 200.0 300.0 ... 9.75e+04 1e+05
  * longitude  (longitude) float32 360B 270.5 271.0 271.5 ... 314.0 314.5 315.0
Attributes:
    long_name:                      Geopotential height @ Isobaric surface
    units:                          gpm
    Grib_Variable_Id:               VAR_0-3-5_L100
    Grib2_Parameter:                [0 3 5]
    Grib2_Parameter_Discipline:     Meteorological products
    Grib2_Parameter_Category:       Mass
    Grib2_Parameter_Name:           Geopotential height
    Grib2_Level_Type:               100
    Grib2_Level_Desc:               Isobaric surface
    Grib2_Generating_Process_Type:  Forecast
    grid_mapping:                   LatLon_361X720-0p25S-180p00E

### Arithmetic

In [33]:
ds['u-component_of_wind_isobaric']

<xarray.DataArray 'u-component_of_wind_isobaric' (time1: 9, isobaric3: 31,
                                                  latitude: 81, longitude: 131)> Size: 12MB
[2960469 values with dtype=float32]
Coordinates:
  * time1      (time1) datetime64[ns] 72B 2017-09-05T12:00:00 ... 2017-09-06T...
    reftime    datetime64[ns] 8B ...
  * latitude   (latitude) float32 324B 50.0 49.5 49.0 48.5 ... 11.0 10.5 10.0
  * isobaric3  (isobaric3) float64 248B 100.0 200.0 300.0 ... 9.75e+04 1e+05
  * longitude  (longitude) float32 524B 250.0 250.5 251.0 ... 314.0 314.5 315.0
Attributes:
    long_name:                      u-component of wind @ Isobaric surface
    units:                          m/s
    Grib_Variable_Id:               VAR_0-2-2_L100
    Grib2_Parameter:                [0 2 2]
    Grib2_Parameter_Discipline:     Meteorological products
    Grib2_Parameter_Category:       Momentum
    Grib2_Parameter_Name:           u-component of wind
    Grib2_Level_Type:               100
    Grib2_Level_Desc:               Isobaric surface
    Grib2_Generating_Process_Type:  Forecast
    grid_mapping:                   LatLon_361X720-0p25S-180p00E

In [34]:
# Using NumPy ufuncs (ex: find wind speed)
np.hypot(ds['u-component_of_wind_isobaric'], ds['v-component_of_wind_isobaric'])

<xarray.DataArray 'u-component_of_wind_isobaric' (time1: 9, isobaric3: 31,
                                                  latitude: 81, longitude: 131)> Size: 12MB
array([[[[11.400518  , 11.300522  , 11.10053   , ...,  3.0067396 ,
           3.0150435 ,  3.0150435 ],
         [11.900499  , 11.800503  , 11.700506  , ...,  4.005077  ,
           4.1110415 ,  4.119547  ],
         [12.301705  , 12.200489  , 12.1004925 , ...,  5.104     ,
           5.1088963 ,  5.115743  ],
         ...,
         [ 5.1087785 ,  5.4671383 ,  5.6727037 , ...,  6.7675405 ,
           7.139995  ,  7.3877916 ],
         [ 5.30373   ,  5.586549  ,  5.86937   , ...,  7.045536  ,
           7.420209  ,  7.6058826 ],
         [ 5.307496  ,  5.7314463 ,  6.0141044 , ...,  7.381703  ,
           7.639993  ,  7.8872957 ]],

        [[ 6.4281073 ,  6.4382105 ,  6.4498487 , ...,  0.8943617 ,
           0.9433309 ,  0.99993163],
         [ 7.2339916 ,  7.1449714 ,  7.1568575 , ...,  0.98486096,
           0.85437185,  0.8245856 ],
         [ 7.840962  ,  7.7414904 ,  7.752462  , ...,  1.5000098 ,
           1.4212738 ,  1.2041636 ],
...
         [ 6.4385486 ,  6.733956  ,  6.9756155 , ...,  2.647466  ,
           2.757438  ,  3.6220796 ],
         [ 6.4771914 ,  6.910966  ,  7.4206753 , ...,  5.2375064 ,
           5.2221446 ,  5.697134  ],
         [ 6.133904  ,  6.3120584 ,  6.8502994 , ...,  4.4847765 ,
           5.2394147 ,  6.012332  ]],

        [[ 5.1994257 ,  5.8487206 ,  5.947258  , ..., 14.007108  ,
          14.007108  , 14.090996  ],
         [ 2.5309954 ,  3.449673  ,  3.4663782 , ..., 13.898929  ,
          13.868762  , 14.090996  ],
         [ 3.6924522 ,  4.5069056 ,  5.31161   , ..., 13.723707  ,
          13.896557  , 14.021022  ],
         ...,
         [ 6.241743  ,  6.4384413 ,  6.6773543 , ...,  2.6475234 ,
           2.6581879 ,  3.495936  ],
         [ 6.206494  ,  6.6428733 ,  7.152106  , ...,  5.2544823 ,
           5.241096  ,  5.7214637 ],
         [ 5.940433  ,  6.0669327 ,  6.643146  , ...,  4.511238  ,
           5.1722403 ,  5.940433  ]]]], dtype=float32)
Coordinates:
  * time1      (time1) datetime64[ns] 72B 2017-09-05T12:00:00 ... 2017-09-06T...
    reftime    datetime64[ns] 8B 2017-09-05T12:00:00
  * latitude   (latitude) float32 324B 50.0 49.5 49.0 48.5 ... 11.0 10.5 10.0
  * isobaric3  (isobaric3) float64 248B 100.0 200.0 300.0 ... 9.75e+04 1e+05
  * longitude  (longitude) float32 524B 250.0 250.5 251.0 ... 314.0 314.5 315.0
Attributes:
    long_name:                      u-component of wind @ Isobaric surface
    units:                          m/s
    Grib_Variable_Id:               VAR_0-2-2_L100
    Grib2_Parameter:                [0 2 2]
    Grib2_Parameter_Discipline:     Meteorological products
    Grib2_Parameter_Category:       Momentum
    Grib2_Parameter_Name:           u-component of wind
    Grib2_Level_Type:               100
    Grib2_Level_Desc:               Isobaric surface
    Grib2_Generating_Process_Type:  Forecast
    grid_mapping:                   LatLon_361X720-0p25S-180p00E

In [35]:
# Using xarray calculations (ex: find mean temperature at each pressure level)
ds['Temperature_isobaric'].mean(('time1', 'latitude', 'longitude'))

<xarray.DataArray 'Temperature_isobaric' (isobaric3: 31)> Size: 124B
array([259.4611 , 255.66313, 248.79195, 240.33334, 235.54622, 231.1286 ,
       223.24205, 218.46536, 213.04253, 208.66364, 204.48088, 209.77391,
       219.08775, 229.3075 , 238.7891 , 247.04654, 253.94908, 259.74255,
       264.77377, 269.32214, 273.50998, 277.24646, 280.57074, 283.58157,
       286.40454, 289.0842 , 291.59814, 292.82648, 294.26868, 295.83304,
       297.45053], dtype=float32)
Coordinates:
    reftime    datetime64[ns] 8B ...
  * isobaric3  (isobaric3) float64 248B 100.0 200.0 300.0 ... 9.75e+04 1e+05

In [36]:
# Perturbation Temperature
ds['Temperature_isobaric'] - ds['Temperature_isobaric'].mean(('time1', 'latitude', 'longitude'))

<xarray.DataArray 'Temperature_isobaric' (time1: 9, isobaric3: 31,
                                          latitude: 81, longitude: 131)> Size: 12MB
array([[[[ -1.4610901 ,  -1.4610901 ,  -1.5610962 , ...,  -1.2610779 ,
           -1.2610779 ,  -1.1611023 ],
         [ -1.5610962 ,  -1.5610962 ,  -1.6611023 , ...,  -1.2610779 ,
           -1.2610779 ,  -1.2610779 ],
         [ -1.6611023 ,  -1.6611023 ,  -1.6611023 , ...,  -1.361084  ,
           -1.361084  ,  -1.361084  ],
         ...,
         [  1.3388977 ,   1.3388977 ,   1.3388977 , ...,  -1.4610901 ,
           -1.361084  ,  -1.2610779 ],
         [  1.5389099 ,   1.5389099 ,   1.5389099 , ...,  -1.361084  ,
           -1.361084  ,  -1.2610779 ],
         [  1.7389221 ,   1.7389221 ,   1.7389221 , ...,  -1.361084  ,
           -1.2610779 ,  -1.1611023 ]],

        [[  0.8368683 ,   0.8368683 ,   0.8368683 , ...,  -0.06312561,
           -0.06312561,  -0.16313171],
         [  0.7368622 ,   0.7368622 ,   0.6368561 , ...,  -0.16313171,
           -0.16313171,  -0.16313171],
         [  0.5368805 ,   0.5368805 ,   0.5368805 , ...,  -0.26313782,
           -0.26313782,  -0.26313782],
...
         [  1.6669617 ,   1.7669678 ,   1.6669617 , ...,   2.0669556 ,
            1.9669495 ,   1.8669739 ],
         [  1.5669556 ,   1.5669556 ,   1.4669495 , ...,   1.5669556 ,
            1.6669617 ,   1.7669678 ],
         [  1.3669739 ,   1.2669678 ,   1.1669617 , ...,   1.1669617 ,
            1.3669739 ,   1.4669495 ]],

        [[ -4.150543  ,  -5.850525  ,  -5.550537  , ..., -10.650543  ,
          -10.550537  , -10.450531  ],
         [ -4.650543  ,  -5.350525  ,  -5.250519  , ..., -10.450531  ,
          -10.550537  , -10.450531  ],
         [ -5.350525  ,  -5.650543  ,  -5.550537  , ..., -10.350525  ,
          -10.150543  ,  -9.950531  ],
         ...,
         [  2.149475  ,   2.2494812 ,   2.2494812 , ...,   2.649475  ,
            2.549469  ,   2.3494568 ],
         [  2.049469  ,   2.049469  ,   1.9494629 , ...,   2.049469  ,
            2.2494812 ,   2.2494812 ],
         [  1.9494629 ,   1.8494568 ,   1.7494812 , ...,   1.7494812 ,
            1.8494568 ,   1.9494629 ]]]], dtype=float32)
Coordinates:
  * time1      (time1) datetime64[ns] 72B 2017-09-05T12:00:00 ... 2017-09-06T...
    reftime    datetime64[ns] 8B 2017-09-05T12:00:00
  * latitude   (latitude) float32 324B 50.0 49.5 49.0 48.5 ... 11.0 10.5 10.0
  * isobaric3  (isobaric3) float64 248B 100.0 200.0 300.0 ... 9.75e+04 1e+05
  * longitude  (longitude) float32 524B 250.0 250.5 251.0 ... 314.0 314.5 315.0

### Documentation

https://xarray.pydata.org/en/stable/index.html